In [4]:
#Import necessary libraries for data manipulation, visualization, and time series analysis.
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from prophet import Prophet

In [ ]:
# Load the NHS A&E time series data from an Excel sheet, skipping the first 14 rows.
# Only specific columns are used and renamed for clarity.
df = pd.read_excel('/Monthly-AE-Time-Series-March-2025.xls',sheet_name='Activity',skiprows=14, usecols="B,C,D,E,F,G", names=['Date', 'Type1', 'Type2', 'Type3', 'Total Attendances', 'Within 4 Hours'])

In [ ]:
print(df.head())

In [ ]:
# Convert date strings to datetime objects
df['Date'] = pd.to_datetime(df['Date'])
# Extract year from date
df['Year'] = df['Date'].dt.year
# Extract month from date
df['Month'] = df['Date'].dt.month
# Create new columns for year, month, and percentage of patients seen within 4 hours.
df['4_hour_target_percentage'] = (df['Within 4 Hours'] / df['Total Attendances']) * 100

In [ ]:
# Check updated dataframe and summary statistics for the new percentage column.
print(df.head())
print(df['4_hour_target_percentage'].describe())

In [ ]:
# Set the 'Date' column as the index and apply seasonal decomposition to identify trend, seasonality, and residuals in the monthly 4-hour target percentages.
yearly_avg = df.groupby('Year')['4_hour_target_percentage'].mean().reset_index()
print(yearly_avg)

monthly_avg = df.groupby('Month')['4_hour_target_percentage'].mean().reset_index()
print(monthly_avg)

df.set_index('Date', inplace=True)
result = seasonal_decompose(df['4_hour_target_percentage'], model='additive', period=12)

result.plot()
plt.show()

In [ ]:
# Prepare the data for Prophet by renaming columns as required: 'ds' for date, 'y' for target value.
df_prophet = df.reset_index()[['Date', '4_hour_target_percentage']].rename(columns={'Date': 'ds', '4_hour_target_percentage': 'y'})
print(df_prophet.head())

In [ ]:
# Initialise and train the Prophet forecasting model.
model = Prophet()
model.fit(df_prophet)

# Create a dataframe that extends 12 months beyond the current data for forecasting.
future = model.make_future_dataframe(periods=12, freq='M')

# Use the trained model to forecast future values.
forecast = model.predict(future)
print(forecast[['ds', 'yhat','yhat_lower', 'yhat_upper']].tail())

In [ ]:
# Plot the forecast and restrict the x-axis range to keep the view focused.
fig = model.plot(forecast)
plt.xlim(['2018-01-01', '2026-12-31'])
plt.show()

In [ ]:
# Save the processed dataset to a CSV file for further use or archiving.
df.reset_index().to_csv('NHS_AE_Cleaned2.csv', index=False)